In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

from __future__ import print_function

import os
from milleniumcohort import create_config
from hsmm4acc import hsmm

import pandas as pd
import numpy as np

In [2]:
config = create_config('../config.yml')
config.create_data_paths()

In [3]:
file_path = config.states_path #config.merged_path
filenames = os.listdir(file_path)
filenames = [fn for fn in filenames if os.path.isfile(os.path.join(file_path,fn))]
filenames = [os.path.join(file_path,filename) for filename in filenames]
print('Processing {} files'.format(len(filenames)))

Processing 1619 files


In [4]:
import pickle

with open(config.model_file, 'rb') as f:
    model = pickle.load(f)

In [63]:
ctab_total = np.zeros((config.hsmmconfig.Nmax,config.hsmmconfig.Nmax))
for fn in filenames[1:2]:
    dataset = pd.read_csv(fn)
    dataset = dataset.set_index('timestamp')
    dataset.index = pd.to_datetime(dataset.index) 
    X = dataset[config.hsmmconfig.column_names].as_matrix()
    prediction = model.predict(X, 0)
    dataset['state_2'] = prediction[1]
    ctab_total += pd.crosstab(dataset['state'], dataset['state_2']).as_matrix()

In [54]:
ctab = pd.crosstab(dataset['state'], dataset['state_2'])
ctab_array = ctab.as_matrix()

In [64]:
ctab_total.diagonal().sum() / float(ctab_array.sum())

0.95194444444444448

In [66]:
pd.DataFrame(ctab_total)

,0,1,2,3,4,5,6,7,8,9
0,106.0,7.0,8.0,13.0,4.0,1.0,7.0,10.0,0.0,3.0
1,7.0,1141.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9.0,0.0,360.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,16.0,0.0,0.0,433.0,2.0,0.0,0.0,1.0,0.0,5.0
4,6.0,0.0,0.0,1.0,225.0,9.0,10.0,0.0,0.0,4.0
5,0.0,0.0,0.0,0.0,1.0,220.0,0.0,0.0,0.0,0.0
6,4.0,0.0,9.0,0.0,4.0,0.0,131.0,0.0,0.0,6.0
7,12.0,0.0,0.0,0.0,0.0,1.0,0.0,479.0,0.0,0.0
8,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,22.0,0.0
9,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,310.0
